In [ ]:
!pip install torch torchvision gradio matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import gradio as gr
import numpy as np
from PIL import Image, ImageOps

In [ ]:
class SimpleMNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        # Corrected input size to the first fully connected layer
        self.fc1 = nn.Linear(64*14*14, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleMNISTCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 20
print_interval = 5

for epoch in range(1, num_epochs + 1):
    model.train()
    correct, total, loss_sum = 0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    # faqat har 10 epochda natijani chiqaramiz
    if epoch % print_interval == 0 or epoch == 1:
        avg_loss = loss_sum / len(train_loader)
        acc = correct / total * 100
        print(f"📘 Epoch {epoch}/{num_epochs}: loss={avg_loss:.4f}, acc={acc:.2f}%")


📘 Epoch 1/20: loss=0.0029, acc=99.92%
📘 Epoch 5/20: loss=0.0033, acc=99.91%
📘 Epoch 10/20: loss=0.0020, acc=99.94%
📘 Epoch 15/20: loss=0.0020, acc=99.94%
📘 Epoch 20/20: loss=0.0021, acc=99.93%


In [ ]:
torch.save(model.state_dict(), "mnist_cnn.pth")

In [ ]:
# --- 2. Bashorat uchun yuklash ---
model = SimpleMNISTCNN()
model.load_state_dict(torch.load("mnist_cnn.pth", map_location="cpu"))
model.eval()

SimpleMNISTCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
import traceback
def predict_with_debug(img):
    try:
        if img is None:
            raise ValueError("Rasm topilmadi — iltimos, chizing va Submit bosing.")

        # --- Sketchpad formatini tozalash ---
        if isinstance(img, dict):
            if "image" in img and img["image"] is not None:
                img = img["image"]
            elif "layers" in img and len(img["layers"]) > 0:
                img = img["layers"][-1]
            else:
                raise ValueError("Sketchpad rasmni yubormadi (bo‘sh).")

        # --- Numpy array -> PIL Image ---
        if isinstance(img, np.ndarray):
            arr = img
            # RGB yoki RGBA formatlarni kulrangga aylantirish
            if arr.ndim == 3:
                if arr.shape[2] == 4:
                    arr = arr[:, :, 3]  # alpha kanal
                else:
                    arr = arr[:, :, 0]  # 1-kanal
            pil = Image.fromarray(arr.astype(np.uint8)).convert("L")
        else:
            pil = img.convert("L")

        # --- 28x28 ga o‘lchamlashtirish ---
        pil = pil.resize((28, 28), Image.LANCZOS)

        # --- Tensor tayyorlash ---
        arr = np.array(pil).astype(np.float32) / 255.0
        arr = (arr - 0.1307) / 0.3081  # MNIST normalizatsiyasi
        x = torch.tensor(arr, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

        # --- Model bashorati ---
        with torch.no_grad():
            logits = model(x)
            probs = F.softmax(logits, dim=1)[0].cpu().numpy()

        return {str(i): float(probs[i]) for i in range(10)}, ""

    except Exception as e:
        import traceback
        print("==== PREDICT ERROR TRACEBACK ====\n", traceback.format_exc())
        return {str(i): 0.0 for i in range(10)}, f"Error: {e}"



# 🔹 Gradio interfeysini yaratish
input_sketchpad = gr.Sketchpad(label="Raqam chizing")
output_label = gr.Label(label="Bashorat")
output_text = gr.Textbox(label="Xatolar")

demo = gr.Interface(
    fn=predict_with_debug,
    inputs=input_sketchpad,
    outputs=[output_label, output_text],
    live=True
)

# 🔹 Gradio ilovasini ishga tushirish
demo.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dbbb2fd73b765ac399.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
